In [1]:
import os
from pathlib import Path
import pandas as pd

import requests

import warnings 
warnings.filterwarnings('ignore') 

import sys
from pathlib import Path
sys.path.insert(0,str(Path(os.path.abspath('.')).parent.parent))
import library.validation.accuracy.metric as metric
import library.validation.accuracy.complexity as complexity

In [2]:
data_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3136_EURAC_shared_folder/data_sites"


In [3]:
sites_metadata = pd.read_csv(os.path.join(data_path, "sites_metadata.csv"), index_col=0)
sites_metadata

,country,latitude,longitude,altitude,provider
station_name,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn
carpentras,France,44.08,5.06,100.0,bsrn
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn
magurele_mars,Romania,44.34,26.01,110.0,bsrn
cabauw,Netherlands,51.97,4.93,0.0,bsrn
...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc
gonzaga,Italy,44.96,10.77,16.0,lombardy


In [4]:
sites_metadata = sites_metadata.iloc[1:5,]

In [6]:
sites_metadata

,country,latitude,longitude,altitude,provider,terrain_classification
station_name,,,,,,
carpentras,France,44.08,5.06,100.0,bsrn,_flat plains
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,_flat plains
magurele_mars,Romania,44.34,26.01,110.0,bsrn,_flat plains
cabauw,Netherlands,51.97,4.93,0.0,bsrn,_flat plains


## Clearness

In [4]:
for index, row in sites_metadata.iterrows():
    site = complexity.SiteComplexity(
        name=index,
        latitude=row['latitude'],
        longitude=row['longitude'],
        altitude=row['altitude']
    )

    # Fetch API data
    site.fetch_api_data()

    # Fetch Clearsky data
    site.fetch_clearsky_data()

    # Calculate clearness
    site.calculate_clearness()

    # Append clearness value to sites_metadata
    if "clearness (%)" not in sites_metadata.columns:
        sites_metadata["clearness (%)"] = None
    sites_metadata.at[index, "clearness (%)"] = site.clearness


## Terrain Classification

In [5]:
for index, row in sites_metadata.iterrows():
    site = complexity.SiteComplexity(name=index, latitude=row['latitude'], longitude=row['longitude'], altitude=row['altitude'])
    
    site.fetch_terrain_classification()
    
    sites_metadata.loc[index, 'terrain_classification'] = site.terrain_classification.value if site.terrain_classification else None


In [6]:
for index, row in sites_metadata.iterrows():
    site = complexity.SiteComplexity(name=index, latitude=row['latitude'], longitude=row['longitude'], altitude=row['altitude'])
    
    site.fetch_tri_1km_value()
    site.fetch_tri_5km_value()
    site.fetch_tri_10km_value()

    site.fetch_tpi_1km_value()
    site.fetch_tpi_5km_value()
    site.fetch_tpi_10km_value()

    sites_metadata.loc[index, 'tri_1km'] = site.tri_1km
    sites_metadata.loc[index, 'tri_5km'] = site.tri_5km
    sites_metadata.loc[index, 'tri_10km'] = site.tri_10km

    sites_metadata.loc[index, 'tpi_1km'] = site.tpi_1km
    sites_metadata.loc[index, 'tpi_5km'] = site.tpi_5km
    sites_metadata.loc[index, 'tpi_10km'] = site.tpi_10km
    

In [6]:
sites_metadata

,country,latitude,longitude,altitude,provider,terrain_classification,tri_1km,tri_5km,tri_10km,tpi_1km,tpi_5km,tpi_10km
station_name,,,,,,,,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn,_flat plains,1.6250,2.875,2.875,0.2500,0.125,0.0
carpentras,France,44.08,5.06,100.0,bsrn,_flat plains,1.5000,2.000,1.375,0.0000,0.000,0.0
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,_flat plains,2.1250,1.375,1.500,0.1875,0.000,0.0
magurele_mars,Romania,44.34,26.01,110.0,bsrn,_flat plains,0.8125,0.750,0.875,0.0000,0.000,0.0
cabauw,Netherlands,51.97,4.93,0.0,bsrn,_flat plains,0.7500,0.500,0.500,0.0625,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc,_flat plains,1.5000,1.250,1.125,-0.1250,0.125,0.0
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc,_flat plains,1.0000,1.125,1.125,0.0625,0.000,0.0
gonzaga,Italy,44.96,10.77,16.0,lombardy,_flat plains,0.8750,1.125,1.000,0.0000,0.000,0.0


## Coastline

In [7]:
for index, row in sites_metadata.iterrows():
    site = complexity.SiteComplexity(name=index, latitude=row['latitude'], longitude=row['longitude'], altitude=row['altitude'])
    site.calculate_distance_to_coastline()
    sites_metadata.at[index, 'distance_to_coastline_km'] = site.distance_to_coastline_km


## Climate Zone

In [8]:
for index, row in sites_metadata.iterrows():
    site = complexity.SiteComplexity(name=index, latitude=row['latitude'], longitude=row['longitude'], altitude=row['altitude'])
    site.fetch_climate_info()

    if site.climate is not None:
        sites_metadata.loc[index,'climate_zone'] = site.climate.value
    else:
        print(f"No climate data available for {index}")

No climate data available for 52.9, 5.38
No climate data available for stavoren


In [5]:
sites_metadata

,country,latitude,longitude,altitude,provider,terrain_classification,tri_1km,tri_5km,tri_10km,tpi_1km,tpi_5km,tpi_10km,distance_to_coastline_km,climate_zone,clearness (%)
station_name,,,,,,,,,,,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn,_flat plains,1.6250,2.875,2.875,0.2500,0.125,0.0,3.211302,Cfb,68.509617
carpentras,France,44.08,5.06,100.0,bsrn,_flat plains,1.5000,2.000,1.375,0.0000,0.000,0.0,57.987479,Cfb,84.703463
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,_flat plains,2.1250,1.375,1.500,0.1875,0.000,0.0,482.028360,Cfb,73.573263
magurele_mars,Romania,44.34,26.01,110.0,bsrn,_flat plains,0.8125,0.750,0.875,0.0000,0.000,0.0,198.059538,Cfa,76.351207
cabauw,Netherlands,51.97,4.93,0.0,bsrn,_flat plains,0.7500,0.500,0.500,0.0625,0.000,0.0,49.955084,Cfb,70.044037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc,_flat plains,1.5000,1.250,1.125,-0.1250,0.125,0.0,4.175609,Cfb,64.296802
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc,_flat plains,1.0000,1.125,1.125,0.0625,0.000,0.0,357.351298,Cfb,71.090746
gonzaga,Italy,44.96,10.77,16.0,lombardy,_flat plains,0.8750,1.125,1.000,0.0000,0.000,0.0,117.377329,Cfa,78.808519


In [6]:
ticket_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis"
sites_metadata.to_csv(os.path.join(ticket_path, 'complexity.csv'))

In [2]:
ticket_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis"
sites_metadata = pd.read_csv(os.path.join(ticket_path, 'complexity.csv'), index_col=0)